# An Introduction to Numpy

## 1. Preface: On Tensors and Arrays

__TL;DR__ Some Deep Learning folks have transformed the term _tensor_ into a fancy buzzword. Do not use it outside its precise scope.

As we are going to see in this lecture, the central concept in NumPy is that of [__ndarray__](https://numpy.org/doc/stable/reference/arrays.ndarray.html#arrays-ndarray). According to NumPy's documentation, an __ndarray__ is _a multi-dimensional container of iterms of same type and size_. In a certain sense, this makes low-level optimization of your code easier.

The fact that NumPy arrays are containers makes it easy to make a parallel between this data type, vectors, matrices and linear transformations. The discussion in this section makes a brief detour around concepts in mathematics, to make it clear that throughout Deep Learning litterature __(almost) anyone is really using the concept of a tensor (!!!)__. Let us begin with some definitions,

__Def 1 (Vector Space).__ A vector space (over $\mathbb{R}$) is a non-empty set $V$ toghether with operations $(+,\cdot)$, called vector addition and multiplication by a scalar, s.t.,

1. __Associativity.__ For $u, v, w \in V$, $u + (v + w) = (u + v) + w$
2. __Commutatitivy.__ For $u, v \in V$, $u + v = v + u$
3. __Identity of Addition.__ $\exists 0 \in V$ s.t. $0 + v = v\text{, } \forall v \in V$
4. __Identity of Scalar Mul.__ $\exists 1 \in V$ s.t. $1 \cdot v = v\text{, } \forall v \in V$
5. __Inverse of Addition.__ $\forall v \in V$, $\exists (-v) \in V$ s.t. $v + (-v) = 0$
6. __Distributivity (addition).__ $a(u + v) = au + av$, $\forall a \in \mathbb{R}$, $\forall u, v \in V$
7. __Distributivity (mul.).__ $(a + b)u = au + bu$, $\forall a, b \in \mathbb{R}$, $\forall u \in V$

__Remark.__ This definition is quite abstract, and comprehends different objects in mathematics. For instance, for $I \subset \mathbb{R}$, the set $C(I) = \{f:I\rightarrow\mathbb{R}/f\text{ is continuous}\}$ is a vector space. On another flavor, $d-$dimensional Euclidean spaces, $\mathbb{R}^{d}$, are vector spaces.

From the above remark, we note that there is a difference between a vector $\mathbf{x} \in \mathbb{R}^{d}$, and the __array__ of its components, i.e., $[x_{1}, x_{2}, \cdots, x_{d}]$. The object is __independent on the choice of basis__, meaning that we can choose whatever linear independent basis for $\mathbf{x}$, it will still behave as a vector (i.e., it will suffice the 7 axioms in Def 1). This is a huge different w.r.t. the array of its components. Especially, the array representation of $\mathbf{x}$, i.e., $[x_{1}, x_{2}, \cdots, x_{d}]$, depends completely on the choice of basis. Usually, when we denote a vector in such a way, we are using the canonical basis, i.e.,

$$\mathbf{x} = \sum_{i=1}^{d}x_{i}\mathbf{e}_{i}$$

where the vector $\mathcal{E} = \{\mathbf{e}_{i}\}_{i=1}^{d}$, $\mathbf{e}_{i} \in \mathbb{R}^{d}$, are such that, $(e_{i})_{j} = \delta(i-j)$

The same remark works for __Linear Transformations__. Recall that a Linear transformation is a function between two vector spaces, that is,

__Def 2 (Linear Transformation)__ Let $V$ and $W$ be two vector spaces over $\mathbb{R}$. A linear transformation is a function $L:V \rightarrow W$ such that,

$$L(au + v) = aL(u) + L(v)$$

__The case of Euclidean Spaces.__ On one hand, this definition of $L$ is __independent__ on the choice of basis on both $V$ and $W$. On the other hand, for fixed basis on $V$ and $W$ [one can associate](https://en.wikipedia.org/wiki/Transformation_matrix) the linear map $L$ to a matrix $\mathbf{L} \in \mathbb{R}^{d_{1} \times d_{2}}$, which explicitly depends on the basis choice.

__Conclusion.__ From this discussion, we have established 2 differences,

- A vector is an element of a vector space. It is an abstract object that obeys certain rules, and is independent on the choice of basis. In an Euclidean setting, the vector is __represented__ through a __1D array of numbers__.
- A linear transform is a function between vector spaces that obeys __linearity__, and is independent on the choice of basis for both its domain and image spaces. In an Euclidean setting, the linear transform is __represented__ through a __2D array of numbers__.

We __do not stop at 1D and 2D__. Usually, in mathematics the concept of __tensor__ is used for __multi-linear relationships__ (i.e., more than 2) that are __independent on the choice of basis__. For a fixed basis, in an Euclidean setting, __a tensor may be represented through a $n-$dimensional array__. The name tensor, widely used in Deep Learning, comes from this association.

__The Problem.__ Usually, if there is no indication that a _tensor treatment_ is necessary (e.g., describing variables in a basis-free framework) one should avoid using the _tensor terminology_. The more correct term would be __n-dimensional array__.

## 2. Going Low Level - Why NumPy?

__Def 3 (Buffer)__ In computer science, a data buffer (or just buffer) is a region of a memory used to store data temporarily while it is being moved from one place to another.

Source: https://en.wikipedia.org/wiki/Data_buffer

NumPy's (Numeric Python) cornerstone is based on __ndarrays__. An __ndarray__ consists on two components,

- The raw data array, called data buffer,
- The information about the data buffer,

The idea of data buffer used in NumPy comes from C. When one __instantiates__ an __ndarray__, one creates a [__contiguous__](https://numpy.org/doc/stable/glossary.html#term-contiguous) and fixed block of memory. The second component contains meta-data on how to interpret the elements in a __ndarray__.

NumPy's implementation comes from C code, which is much more performant than pure Python. From a low level perspective, all __ndarrays__ are seen as "chunks" of memory, which are acessed using strides. For instance, one can make an analogy to [row and column-major orderings of matrices](https://en.wikipedia.org/wiki/Row-_and_column-major_order).

### 2.1. An Introductory Example

In this preliminary example, we are going to explore a Random Walk. This corresponds to Sec. 2.1 of [From Python to NumPy](https://www.labri.fr/perso/nrougier/from-python-to-numpy/).

__Def 3 (Discrete Random Walk)__ Let $n_{0} \in \mathbb{Z}$. A random walk is a sequence of steps $\{n_{t}\}_{t=1}^{T}$ such that,

$$
n_{t+1}=\begin{cases}
n_{t} + 1&\text{w/ prob. }0.5,\\
n_{t} - 1&\text{w/ prob. }0.5,
\end{cases}
$$

In [3]:
import random
import numpy as np
from itertools import accumulate

In [9]:
%%timeit

n0 = 0
n = 0

random_walk = []
for _ in range(1000):
    n = n + (2 * random.randint(0, 1) - 1)
    random_walk.append(n)

346 µs ± 757 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


we can improve the execution of this code by noting the following,

$$n_{t+1} = n_{t} + \epsilon_{t}$$

where $\epsilon_{t} = choice(-1, 1)$. As a consequence, we may __unwrap__ the temporal variable:

$$n_{t+1} = n_{t} + \epsilon_{t} = n_{t-1} + \epsilon_{t-1} + \epsilon_{t}$$

we may repeat this until we reach $n_{0}$,

$$n_{t+1} = n_{0} + \sum_{i=0}^{t}e_{i}$$

__if we know in advance__ the sequence $\{e_{0},\cdots,e_{t}\}$, we can use the following,

In [9]:
%%timeit

n0 = 0

# Second implementation: accumulation
steps = random.choices([-1, 1], k=1000)
random_walk = [0] + list(accumulate(steps))

60.7 µs ± 515 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Finally, we can repeat all of these using NumPy's Cumsum function,

In [10]:
%%timeit

n0 = 0

# Third implementation: cumsum
steps = np.random.choice([-1, 1], size=1000)
random_walk = n0 + np.cumsum(steps)

10.8 µs ± 17.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### 2.2. NumPy Arrays (ndarrays)

In [2]:
import numpy as np

The most basic declaration of a numpy array is through a list. For instance,

In [11]:
a = np.array([1, 2, 3, 4])
a

array([1, 2, 3, 4])

in this case, you can index the numpy array like a list,

In [4]:
print(a[0], a[1], a[2])

1 2 3


as mentioned previously, numpy arrays have a fixed data type. In the previous case we declared an array of ints,

In [5]:
a.dtype

dtype('int64')

here, we're using 64 bit integers as the data type of our array. Other data types (dtypes) are supported as well. You can cast a numpy array with,

In [9]:
a.astype(np.float32)

array([1., 2., 3., 4.], dtype=float32)

note that this operation __creates__ a new array. You can check that below,

In [10]:
a

array([1, 2, 3, 4])

when possible, numpy will cast your data to float,

In [12]:
a = np.array([0, 1., 2, 3.])
a

array([0., 1., 2., 3.])

as well as complex,

In [13]:
a = np.array([0., 1.+1.j])
a

array([0.+0.j, 1.+1.j])

In [14]:
a.dtype

dtype('complex128')

unlike pure Python (that does not support casting complex numbers), you can cast complex back into float,

In [15]:
a.astype(np.float32)

/tmp/ipykernel_5661/309460260.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  a.astype(np.float32)


array([0., 1.], dtype=float32)

#### 2.2.1. Creating NumPy Arrays

Besides creating arrays directly from lists, a few functions are useful in certain contexts. For instances, you can create an array with 0 in all its entries like this,

In [3]:
x = np.zeros(5)
x

array([0., 0., 0., 0., 0.])

the argument of this function gives us the __shape__ of the array. You could, for instance, create a matrix in such a way,

In [5]:
x = np.zeros([5, 5])
x

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

__Note on syntax.__ the function ```np.zeros``` receives a scalar or an iterable as the shape argument. As a consequence you need to pass a list or a tuple for specifying the shape of the array.

Of course, you can create 3D arrays in the same way,

In [6]:
x = np.zeros([2, 5, 5])
x

array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])

By default, NumPy arrays have fixed size. This means that you cannot append elements dynamically to an existing array. This is the opposite w.r.t. Python lists. You can manipulate existing arrays with ```np.concatenate```. For instance,

In [7]:
a = np.array([0, 1, 2])
b = np.array([3, 4, 5])
c = np.concatenate([a, b])

print(c)
print(id(a), id(b), id(c))

[0 1 2 3 4 5]
139669163422032 139669163422608 139669163425296


note that by concatenating arrays you create a new place in memory.

### 2.3. Indexing

This part of this lecture is based on [NumPy's documentation on indexing and slicing](https://numpy.org/doc/stable/user/basics.indexing.html).

#### 2.3.1. Single Element Indexing

As we discussed in this lecture, NumPy arrays can be "n-"dimensional, in the sense that you can create an object indexed by $i_{1}, i_{2},\cdots,i_{n}$. For instance,

In [21]:
v = np.array([1., 0., 0.])
A = np.array([[1., 0., 0.],
              [0., 1., 0.],
              [0., 0., 1.]])

print(v)
print(A)

[1. 0. 0.]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


to declare a one-dimensional vector (indexed through $i$), we use a list. For a 2 dimensional matrix, we use a __list of lists__. This allows us to do an easier indexing,

In [22]:
print(A[0, 0], A[1, 1], A[2, 2])

1.0 1.0 1.0


note that with pure Python, such an indexing is not possible,

In [23]:
A = [[1., 0., 0.],
     [0., 1., 0.],
     [0., 0., 1.]]

print(A[0, 0])

TypeError: list indices must be integers or slices, not tuple

furthermore, you can index numpy arrays with fewer indices,

In [26]:
A = np.array([[1., 3., 3.],
              [3., 1., 5.],
              [5., 5., 1.]])

print(A[0])

[1. 3. 3.]


which gives you the first row of the array. Conversely, you could choose the first column like that,

In [27]:
print(A[:, 0])

[1. 3. 5.]


The colon $:$ serves to indicate that we take every element along the 1st dimension. Otherwise you could use transposition,

In [28]:
print(A.T[0])

[1. 3. 5.]


every NumPy array has an attribute called __shape__. The shape gives you the format of the array. For instance,

In [24]:
# Samples elements using a normal distribution
A = np.random.randn(1000, 2)

print(A.shape)

(1000, 2)


In this case we generated a random $(1000, 2)$ matrix. This matrix has $1000$ rows and $2$ columns.

#### 2.3.2. Slicing and Striding

NumPy's slicing/striding extends these concepts from pure Python to n-dimensional arrays. Throughout these examples we assume $\mathbf{A}$ has $n$ dimensions, that is, it has shape $(d_{1},d_{2},\cdots,d_{n})$.

For a 1 dimensional array, slicing consists on __choosing a slice__ out of the array $\mathbf{A} = [a_{1},\cdots,a_{d}]$. This means that for $0 \leq i_{1} \leq i_{2} < d$, we choose $[a_{i_{1}},\cdots,a_{i_{2}}]$ out of $[a_{1},\cdots,a_{d}]$. As in pure Python, negative indices are interpreted as counting backwards. For instance,

In [30]:
# Creates an array with 0, ..., 9
A = np.arange(0, 10)
print(A[1:7])

[1 2 3 4 5 6]


As in pure Python, you can also stride through the array. The complete syntax for slicing arrays consists of $i:j:k$, where $i$ is the starting index, $j$ is the stopping index, and $k$ is the stride/step. This will select elements $i,i+k,\cdots,i+(m-1)k$.

Let $q$ and $r$ be the quotient and remainder of the division $j-i$ by $k$, i.e., $j-i = qk + r$. The value $m$ is given by $m = q + (r \neq 0)$. In the same idea of the last example,

In [31]:
print(A[1:7:2])

[1 3 5]


__Remark.__ Ignoring the start index results in NumPy considering the slice from the beginning. Conversely, ignoring the start index results in NumPy considering the slice up to the end of the array, i.e.,

In [32]:
print(A[:7], A[7:])

[0 1 2 3 4 5 6] [7 8 9]


__Remark.__ Slicing/Striding produces a view instead of a copy. You should be aware of that when perfoming operations on sub-arrays, especially if you assign a sub-array to a variable. For instance, let us extract the even rows of the following matrix,

In [97]:
A = np.arange(0, 10).reshape(5, 2)

print(A)

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]


In [98]:
sub_A = A[::2]

In [99]:
print(A)
print(sub_A)

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
[[0 1]
 [4 5]
 [8 9]]


__Warning.__ it may not be straightfoward to see if two arrays have the same reference, because NumPy arrays are effectively pointers to places in memory. In this sense the pointer makes a reference to the starting point of memory. As such, they can point to different, but overlapping places. Check the following,

In [100]:
id(A), id(sub_A)

(140545971173488, 140545971125584)

which are different. But you can check that the two arrays share memory,

In [101]:
np.shares_memory(A, sub_A)

True

and indeed, if you do,

In [102]:
sub_A[0] = -1

In [103]:
sub_A

array([[-1, -1],
       [ 4,  5],
       [ 8,  9]])

In [104]:
A

array([[-1, -1],
       [ 2,  3],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9]])

conversely, changing $A$ yields a change in the sub-array,

In [105]:
A[2] = -5

In [106]:
sub_A

array([[-1, -1],
       [-5, -5],
       [ 8,  9]])

In [107]:
A

array([[-1, -1],
       [ 2,  3],
       [-5, -5],
       [ 6,  7],
       [ 8,  9]])

__Jargon.__ In this case, we say that indexing generates __a view__ of the array $A$. Modifying the array modifies the view, and modifying the view modifies the array. A way to avoid this tie is to use ```.copy()```. This way a new place in memory will be created for the view,

In [108]:
A = np.arange(0, 10).reshape(5, 2)
sub_A = A[::2].copy()

print(np.shares_memory(A, sub_A))

False


#### 2.3.3. Dimensional Indexing

In a multi-dimensional array, you can use Ellipsis, $\cdots$, so that NumPy automatically expands $:$ needed for the selection tuple to index all dimensions. For instance,

In [110]:
A = np.random.randn(3, 224, 224)

print(A[..., 0].shape)

(3, 224)


which is equivalent to,

In [111]:
print(A[:, :, 0].shape)

(3, 224)


In addition, you can create new dimensions on numpy arrays using ```np.newaxis``` or ```None```,

In [10]:
a = np.ones([5, 5])
b = np.zeros([5, 5])

print(a[None, ...].shape)
print(b[..., None].shape)

(1, 5, 5)
(5, 5, 1)


In [11]:
a = np.ones([5, 5])
b = np.zeros([5, 5])

print(a[np.newaxis, ...].shape)
print(b[..., np.newaxis].shape)

(1, 5, 5)
(5, 5, 1)


this may be useful when one wants to concatenate arrays over new dimensions,

In [13]:
a = np.ones([5, 5])
b = np.zeros([5, 5])
c = np.concatenate([a[None, ...], b[None, ...]], axis=0)

c

array([[[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])

### 2.4. Vectorization

In [6]:
A = np.ones([100, 100])
v = np.arange(100)

print(A.shape, v.shape)

(100, 100) (100,)


In [7]:
%%timeit

u = np.zeros_like(v)
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        u[i] = A[i, j] * v[j]

1.56 ms ± 41.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit

u = np.dot(A, v)

1.22 µs ± 16.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### 2.5. Broadcasting

From NumPy's documentation,

> The term broadcasting describes how NumPy treats arrays with different shapes during arithmetic operations. Subject to certain constraints, the smaller array is “broadcast” across the larger array so that they have compatible shapes. Broadcasting provides a means of vectorizing array operations so that looping occurs in C instead of Python. It does this without making needless copies of data and usually leads to efficient algorithm implementations. There are, however, cases where broadcasting is a bad idea because it leads to inefficient use of memory that slows computation.

so, sometimes you want to add or multiply arrays over an existing dimension, or over a new dimension. Instead of doing a for loop, the optimized way of doing so is through broadcasting. Here are a few examples,

In [14]:
a = np.array([1, 2, 3])
b = 3
c = a + b

print(c)

[4 5 6]


here, the scalar ```3``` is stretched over the shape of ```a```, being added to each element of the array. A slightly more complex example,

In [18]:
A = np.zeros([5, 5])
b = np.arange(5)

print(A)
print(b)
print(A + b)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[0 1 2 3 4]
[[0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]]


As you can see, the array ```b``` was stretched along a second dimension. This creates a new matrix,

```
[[0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4]]
```

which is then added to the matrix of zeroes.

__Warning.__ The broadcasting occurs w.r.t. the dimension not present in the smaller array. In our previous example, ```b``` has shape $(5,)$, which means that it will be repeated accross __the first dimension__. Compare the result with the following block of code, 

In [19]:
A = np.zeros([5, 5])
b = np.arange(5)[:, None]

print(A)
print(b)
print(A + b)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0]
 [1]
 [2]
 [3]
 [4]]
[[0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2.]
 [3. 3. 3. 3. 3.]
 [4. 4. 4. 4. 4.]]


now, the array ```b``` is repeated __along the second dimension__, i.e., $(5,1) \mapsto (5, 5)$.

__The rule.__ According to [NumPy's documentation](https://numpy.org/doc/stable/user/basics.broadcasting.html), when operating two arrays, NumPy compares the dimensions, from __right to the left__. Two dimensions are compatible __if__,

1. They are equal, or
2. one of them is 1

In the second case, let's say $b$ has $1$ along a given dimension. NumPy repeats the array along this dimension in order to match $a$.